<a href="https://colab.research.google.com/github/sycho2003/20252R0136COSE36203/blob/main/alternative_thought_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 이것저것 설치


정상적으로 실행하려면 내드라이브에

https://drive.google.com/drive/folders/17Hy2LFO4j6IZiwZG2jDoC0lxwF2NPLGR

https://drive.google.com/drive/folders/1Vn0qXdF2O5YuEk_CmdE720MzxnnDbxDf

위의 두 드라이브의 바로가기를 만들어둬야함

In [ ]:
pip install -U llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121


Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 863.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 16.6 MB/s eta 0:00:00


## 세팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# 1) LoRA+DANN 이진 분류 모델이 들어있는 폴더 (saved_lora_dann_model)
LORA_DANN_DIR = "/content/drive/MyDrive/LoRA+DaNN 이진분류 모델 best/saved_lora_dann_model"

# 2) 왜곡 종류별 이진 분류 모델이 들어있는 폴더 (model1~12_weights.pth)
SMOTE_LORA_DIR = "/content/drive/MyDrive/SMOTE+LoRA"


In [ ]:
from llama_cpp import Llama

REPO_ID = "hugging-quants/Llama-3.2-3B-Instruct-Q4_K_M-GGUF"
FILENAME = "*q4_k_m.gguf"

llm = Llama.from_pretrained(
    repo_id=REPO_ID,
    filename=FILENAME,
    n_ctx=4096,
    n_batch=512,
    n_threads=8,
    n_gpu_layers=-1,
    verbose=True,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


./llama-3.2-3b-instruct-q4_k_m.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (Tesla T4) - 14992 MiB free
llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /root/.cache/huggingface/hub/models--hugging-quants--Llama-3.2-3B-Instruct-Q4_K_M-GGUF/snapshots/eb72f2a08dd2b9edd07ffacfe5aa56938b7939b0/./llama-3.2-3b-instruct-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                    

In [ ]:
import pandas as pd
data1 = pd.read_csv('d01_preprocessed_revised.csv')
data2 = pd.read_csv('d02_preprocessed.csv')
data3 = pd.read_csv('d03_preprocessed.csv')

In [ ]:
data3_sampled = data3.groupby('situation').sample(n=1, random_state=42)

In [ ]:
data1["reframe_type"] = "self"
data3["reframe_type"] = "self"
data2["reframe_type"] = "counselor"

df = pd.concat([data1, data2, data3], ignore_index=True)

In [ ]:
DISTORTION_COLS = [
    "all-or-nothing thinking",
    "comparing and despairing",
    "disqualifying the positive",
    "emotional reasoning",
    "fortune telling",
    "labeling",
    "magnification",
    "mind reading",
    "overgeneralizing",
    "should statements",
    "mental filter",
    "personalization and blaming",
]


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from peft import LoraConfig, get_peft_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 코퍼스 만들기

In [ ]:
import pandas as pd

def row_labels(row) -> list[str]:
    labels = []
    for c in DISTORTION_COLS:
        v = row.get(c, 0)
        try:
            if int(v) == 1:
                labels.append(c)
        except:
            pass
    # has_distortion=0인데 원핫이 비어있지 않는 데이터가 있을 수도 있어서
    # 최종 라벨이 비면 "none"으로 처리
    return labels

def build_corpus_from_df(df: pd.DataFrame):
    corpus = []
    for _, r in df.iterrows():
        situation = "" if pd.isna(r["situation"]) else str(r["situation"]).strip()
        thought    = "" if pd.isna(r["thought"])    else str(r["thought"]).strip()
        reframe    = "" if pd.isna(r["reframe"])    else str(r["reframe"]).strip()
        if not thought or not reframe:
            continue

        # 원핫 라벨 뽑기는  row_labels 그대로 쓰면 됨
        labels = row_labels(r)
        has_dist = int(r.get("has_distortion", 0)) if not pd.isna(r.get("has_distortion", 0)) else 0

        reframe_type = r.get("reframe_type", "unknown")
        if pd.isna(reframe_type):
            reframe_type = "unknown"

        search_text = f"Situation: {situation}\nThought: {thought}".strip()

        corpus.append({
            "search_text": search_text,
            "reframe": reframe,
            "meta": {
                "has_distortion": bool(has_dist),
                "labels": labels if labels else (["none"] if has_dist == 0 else ["unknown"]),
                "reframe_type": reframe_type,
                "situation": situation,
                "thought": thought,
            }
        })
    return corpus




corpus = build_corpus_from_df(df)
len(corpus), corpus[0]

(2487,
 {'search_text': 'Situation: A Roomate of mine stole my comptuer\nThought: Someone I trusted stole something valuable of mine, I was extremely angry and wanted justice',
  'reframe': 'My roommate stole something of mine, and I will focus on actionable solutions to address this.',
  'meta': {'has_distortion': True,
   'labels': ['emotional reasoning'],
   'reframe_type': 'self',
   'situation': 'A Roomate of mine stole my comptuer',
   'thought': 'Someone I trusted stole something valuable of mine, I was extremely angry and wanted justice'}})

In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# (이미 설치돼있다면 OK)
embedder = SentenceTransformer("intfloat/multilingual-e5-small")

def embed_texts(texts):
    emb = embedder.encode(texts, normalize_embeddings=True, show_progress_bar=True)
    return np.array(emb, dtype=np.float32)

# 1) corpus의 search_text를 임베딩
passages = [f"passage: {d['search_text']}" for d in corpus]
X = embed_texts(passages)

# 2) FAISS 인덱스 생성 (cosine 유사도 == IP with normalized embeddings)
index = faiss.IndexFlatIP(X.shape[1])
index.add(X)

print("index size:", index.ntotal)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/78 [00:00<?, ?it/s]

index size: 2487


## 모델 불러오기

In [ ]:
# Gradient Reversal (DANN용)
class GradientReversalFn(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambda_val):
        ctx.lambda_val = lambda_val
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambda_val * grad_output, None

class GradientReversalLayer(nn.Module):
    def __init__(self, lambda_val=1.0):
        super().__init__()
        self.lambda_val = lambda_val

    def forward(self, x):
        return GradientReversalFn.apply(x, self.lambda_val)


class LoRADANNClassifier(nn.Module):
    def __init__(self, feature_extractor, num_classes, num_domains, lambda_val=1.0):
        super().__init__()
        self.feature_extractor = feature_extractor
        hidden = feature_extractor.config.hidden_size

        self.label_classifier = nn.Sequential(
            nn.Linear(hidden, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
        self.domain_classifier = nn.Sequential(
            GradientReversalLayer(lambda_val),
            nn.Linear(hidden, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_domains),
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.feature_extractor(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        features = outputs.last_hidden_state[:, 0, :]  # [CLS]
        label_logits = self.label_classifier(features)
        domain_logits = self.domain_classifier(features)
        return label_logits, domain_logits


In [ ]:
class InnerProductClassifier(nn.Module):
    def __init__(self, input_dim, label_embeddings, trainable_label_emb: bool = True):
        super().__init__()
        # 1) 입력 피처를 label embedding 차원으로 projection
        self.proj = nn.Linear(input_dim, label_embeddings.size(1))

        # 2) 라벨 임베딩 (trainable 여부 선택)
        if trainable_label_emb:
            # 학습 가능한 파라미터
            self.label_emb = nn.Parameter(label_embeddings.clone())
        else:
            # 학습 동안 고정
            self.register_buffer("label_emb", label_embeddings.clone())

    def forward(self, x):
        # x: (batch, input_dim)
        x_proj = self.proj(x)                      # (batch, emb_dim)
        logits = torch.matmul(x_proj, self.label_emb.T)  # (batch, num_labels)
        return logits


In [ ]:
import re, string

def normalize_text(s: str) -> str:
    def white_space_fix(text):
        return " ".join(text.split())

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


In [ ]:
from transformers import BertTokenizer

# 토크나이저는 saved_lora_dann_model 폴더에 같이 저장되어 있음
tokenizer = BertTokenizer.from_pretrained(LORA_DANN_DIR)

# 베이스 BERT
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_model.to(device)

# 학습 때 썼던 LoRA 설정 (r=8, alpha=16, dropout=0.1)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="FEATURE_EXTRACTION",
)

bert_model_with_lora = get_peft_model(bert_model, lora_config)

# DANN 모델 인스턴스 (task: 2클래스, domain: 3개였음) :contentReference[oaicite:6]{index=6}
num_task_labels = 2
num_domains = 3
lora_dann_model = LoRADANNClassifier(
    feature_extractor=bert_model_with_lora,
    num_classes=num_task_labels,
    num_domains=num_domains,
)

state_dict = torch.load(
    f"{LORA_DANN_DIR}/lora_dann_model_weights.pth",
    map_location=device,
)
lora_dann_model.load_state_dict(state_dict)

lora_dann_model.to(device)
lora_dann_model.eval()

# feature_extractor (BERT+LoRA) 는 아래에서 왜곡별 이진 분류에도 같이 사용
feature_extractor = lora_dann_model.feature_extractor
hidden_size = feature_extractor.config.hidden_size


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
DISTORTION_LABELS = [
    "all-or-nothing thinking",
    "comparing and despairing",
    "disqualifying the positive",
    "emotional reasoning",
    "fortune telling",
    "labeling",
    "magnification",
    "mind reading",
    "overgeneralizing",
    "should statements",
    "mental filter",
    "personalization and blaming",
]

In [ ]:
binary_label_classifiers = []

for i in range(1, 13):
    # label_emb는 load_state_dict로 덮어씌워질 거라 shape만 맞으면 됨
    init_label_emb = torch.randn(2, hidden_size)
    clf = InnerProductClassifier(hidden_size, init_label_emb)

    weight_path = f"{SMOTE_LORA_DIR}/model{i}_weights.pth"
    clf.load_state_dict(torch.load(weight_path, map_location=device))

    clf.to(device)
    clf.eval()
    binary_label_classifiers.append(clf)

assert len(binary_label_classifiers) == len(DISTORTION_LABELS)


In [ ]:
def classify_distortion(
    thought_text: str,
    threshold: float = 0.77,
    top_k: int = 3,
) -> dict:
    """
    thought_text: 입력 자동적 사고 텍스트
    threshold: p(distorted) 기준 (0~1)
    top_k: 상위 몇 개 왜곡 라벨을 반환할지
    """
    # 1) 왜곡 여부 (LoRA+DANN 이진 분류)
    encoded = tokenizer(
        normalize_text(thought_text),
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512,
    )

    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)

    with torch.no_grad():
        label_logits, _ = lora_dann_model(input_ids, attention_mask)
        label_probs = torch.softmax(label_logits, dim=-1)[0]  # shape: [2]

        prob_not_distorted = float(label_probs[0].item())
        prob_distorted     = float(label_probs[1].item())
        is_distorted = prob_distorted >= threshold

        if not is_distorted:
            return {
                "is_distorted": False,
                "prob_not_distorted": prob_not_distorted,
                "prob_distorted": prob_distorted,
                "top_labels": [],          # 왜곡 종류 아예 계산 안 함
                "conf_distortion": None,   # 최대 왜곡 강도도 없음
            }

        # 2) [CLS] 임베딩 추출 (같은 feature_extractor 재사용)
        feat_outputs = feature_extractor(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        cls_emb = feat_outputs.last_hidden_state[:, 0, :]  # (1, hidden_size)

        # 3) 왜곡 종류별 이진 분류기 12개 돌리기
        label_scores = {}
        for name, clf in zip(DISTORTION_LABELS, binary_label_classifiers):
            logits = clf(cls_emb)              # (1, 2)
            probs = torch.softmax(logits, -1)[0]
            label_scores[name] = float(probs[1].item())  # label=1 (해당 왜곡 있음) 확률

    # 4) 상위 top_k 라벨 정렬
    sorted_labels = sorted(
        label_scores.items(),
        key=lambda x: x[1],
        reverse=True,
    )
    top_labels = sorted_labels[:top_k]

    return {
        "is_distorted": is_distorted,
        "prob_distorted": prob_distorted,
        "prob_not_distorted": prob_not_distorted,
        "label_scores": label_scores,   # 전체 dict: {라벨명: p}
        "top_labels": top_labels,       # [(라벨명, p), ...]
    }


## 대안적 사고 생성

In [ ]:
def retrieve_two_styles(
    thought_text: str,
    cls: dict,
    top_k_self=3,
    top_k_counselor=3,
    widen=80,
):
    is_distorted = bool(cls.get("is_distorted", True))

    # 예측 라벨들 (멀티라벨 top-k)
    pred_labels = [lab for lab, _ in (cls.get("top_labels") or [])]
    pred_set = set(pred_labels)

    # 핵심: 왜곡이 아니면 라벨 기반 쿼리를 약하게(라벨 제거)
    if not is_distorted:
        query = f"query: thought={thought_text}"
        pred_set = set()  # match_count 의미 없게 처리
    else:
        labels_for_query = ", ".join(pred_labels) if pred_labels else "unknown"
        query = f"query: labels={labels_for_query} | thought={thought_text}"

    q = embed_texts([query])
    scores, idxs = index.search(q, widen)

    cands = []
    for i, s in zip(idxs[0], scores[0]):
        if i == -1:
            continue

        item = corpus[i]
        item_labels = set(item["meta"].get("labels", []))
        reframe_type = item["meta"].get("reframe_type", "unknown")

        match_count = len(pred_set & item_labels) if pred_set else 0

        cands.append({
            "score": float(s),
            "match_count": match_count,     # 왜곡 False면 항상 0
            "reframe_type": reframe_type,
            "support": item["reframe"],
            "meta": item["meta"],
        })

    # 정렬 로직도 살짝 다르게:
    # - 왜곡 True: 라벨 매치 우선 → 유사도
    # - 왜곡 False: 유사도만 우선
    if is_distorted:
        cands.sort(key=lambda x: (x["match_count"], x["score"]), reverse=True)
    else:
        cands.sort(key=lambda x: x["score"], reverse=True)

    self_bucket = [x for x in cands if x["reframe_type"] == "self"][:top_k_self]
    counselor_bucket = [x for x in cands if x["reframe_type"] == "counselor"][:top_k_counselor]

    # if len(self_bucket) < top_k_self:
    #     filler = [x for x in cands if x not in self_bucket][: (top_k_self - len(self_bucket))]
    #     self_bucket += filler
    #
    # if len(counselor_bucket) < top_k_counselor:
    #     filler = [x for x in cands if x not in counselor_bucket][: (top_k_counselor - len(counselor_bucket))]
    #     counselor_bucket += filler

    return self_bucket, counselor_bucket


In [ ]:
def build_messages_two_versions(
    thought_text: str,
    cls: dict,
    self_docs: list[dict],
    counselor_docs: list[dict],
):
    # 1) RAG 예시 텍스트 클리닝: 다른 사연/헤더/지시문 제거
    def clean_support(text: str) -> str:
        """
        RAG 예시 중에서 [USER], [SYSTEM] 같은 헤더나
        'Please provide...', 'Your turn' 같은 워크북/프롬프트 문장을 제거하고,
        실제 대안적 사고 문장만 남긴다.
        """
        if not isinstance(text, str):
            return ""
        lines = []
        bad_markers = [
            "[USER]", "[SYSTEM]", "[ORIGINAL THOUGHT]", "[DISTORTION CLASSIFICATION]",
            "[OUTPUT]", "[REPLACEMENT OF DISTORTION]", "[NEW DISTORTION]",
            "Please provide", "Please replace", "Please fill", "Your turn",
        ]
        for line in text.splitlines():
            s = line.strip()
            if not s:
                continue
            # 대괄호 헤더류는 버리기
            if s.startswith("[") and s.endswith("]"):
                continue
            # 워크북/지시문/메타 문장 제거
            if any(m.lower() in s.lower() for m in bad_markers):
                continue
            lines.append(s)
        return " ".join(lines)

    # 2) 클리닝 적용해서 RAG 예시 만들기
    self_lines = []
    for d in self_docs:
        txt = clean_support(d.get("support", ""))
        if txt:
            self_lines.append(f"- {txt}")
    counselor_lines = []
    for d in counselor_docs:
        txt = clean_support(d.get("support", ""))
        if txt:
            counselor_lines.append(f"- {txt}")

    self_refs = "\n".join(self_lines) or "- (없음)"
    counselor_refs = "\n".join(counselor_lines) or "- (없음)"

    # 멀티라벨 요약 문자열
    top_labels = cls.get("top_labels") or []
    if top_labels:
        labels_line = ", ".join(
            [f"{lab}({p:.2f})" for lab, p in top_labels if isinstance(p, (int, float))]
        )
        if not labels_line:
            labels_line = ", ".join([str(lab) for lab, _ in top_labels])
    else:
        labels_line = "unknown"

    conf_dist = cls.get("conf_distortion")
    conf_line = (
        f"\n- 왜곡 강도(최대 확률): {conf_dist:.2f}"
        if isinstance(conf_dist, (int, float))
        else ""
    )

    system = (
        "Do NOT write any preface, explanation, or notes.\n"
        "Output ONLY the two sections in the exact OUTPUT FORMAT.\n\n"
        "You are a CBT-based assistant.\n"
        "Your task is to REFRAME the ORIGINAL THOUGHT into two short versions:\n"
        "- SELF_VERSION: first person, using 'I'.\n"
        "- COUNSELOR_VERSION: second person, using 'you'.\n\n"
        "Use the RAG EXAMPLES only as references for style and ideas.\n"
        "Do NOT copy their wording, and do NOT bring in their situations.\n"
        "Stay strictly on the topic of the ORIGINAL THOUGHT (no new themes like family,\n"
        "gender, relationships, diagnoses, etc. unless they already appear in the original).\n\n"
        "The SELF, COUNSELOR version must be more balanced and compassionate than the original thought,\n"
        "and must NOT become more extreme or hopeless (no 'I should give up', 'there is no hope', etc.).\n"
        "Write natural, realistic, and kind sentences. Avoid absolute words like 'always' or 'never'.\n"
        "Keep each version as one short paragraph.\n\n"
        "OUTPUT FORMAT (must be exact):\n"
        "SELF_VERSION:\n"
        "<one paragraph of first-person alternative thought>\n\n"
        "COUNSELOR_VERSION:\n"
        "<one paragraph of second-person counselor-style alternative thought>\n"
    )


    # --- 4) 유저 프롬프트 (분류 정보 + RAG 예시, 영어) ---
    user = f"""[ORIGINAL THOUGHT]
{thought_text}

[DISTORTION CLASSIFICATION]
- Is distorted: {cls.get("is_distorted")}
- Top candidate distortions: {labels_line}{conf_line}

[RAG EXAMPLES - SELF REFRAME (first person)]
{self_refs}

[RAG EXAMPLES - COUNSELOR ADVICE (second person)]
{counselor_refs}
"""

    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]


In [ ]:
def generate_with_llama(messages, max_tokens: int = 512, temperature: float = 0.7, top_p: float = 0.9):
    """
    대괄호 태그([SYSTEM]/[USER]) 쓰면 모델이 출력에서도 대괄호를 흉내내는 경우가 많아서
    SYSTEM/USER 라벨만 쓰는 형태로 변경.
    """
    parts = []
    for m in messages:
        role = m.get("role", "")
        content = m.get("content", "")
        if role == "system":
            parts.append(f"SYSTEM:\n{content}\n")
        else:
            parts.append(f"USER:\n{content}\n")

    prompt = "\n".join(parts) + "\nASSISTANT:\n"

    out = llm(
        prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        # 있으면 도움 됨 (llama-cpp에 stop 지원하는 경우)
        stop=["\nSYSTEM:", "\nUSER:", "\nASSISTANT:", "\nNote:", "\nNOTE:"],
    )
    return out["choices"][0]["text"].strip()



In [ ]:
import re

def clean_version_text(s: str) -> str:
    if not isinstance(s, str):
        return ""

    t = s.strip()

    # "Note:" 이후는 통째로 제거 (혹시 버전 안에 섞였을 때 대비)
    t = re.split(r"\n\s*(?:Note|NOTE)\s*:", t, maxsplit=1)[0].strip()

    # 양끝에 튀는 대괄호/괄호/따옴표 찌꺼기 제거
    t = t.strip(" \t\r\n[](){}<>\"'")

    # 앞뒤에 홀로 남는 ] [ 같은 토큰 제거
    t = re.sub(r"^\s*[\]\[]+\s*", "", t)
    t = re.sub(r"\s*[\]\[]+\s*$", "", t)

    # 줄바꿈/여백 정리 → 한 단락
    lines = [ln.strip() for ln in t.splitlines() if ln.strip()]
    t = " ".join(lines).strip()

    # 또 한 번 안전하게 양끝 정리
    return t.strip(" \t\r\n[](){}<>\"'")

def _cut_from_last_self_version(text: str) -> str:
    if not isinstance(text, str):
        return ""
    idx = text.rfind("SELF_VERSION")
    return text[idx:].strip() if idx != -1 else text.strip()

def _squash(s: str) -> str:
    if not isinstance(s, str):
        return ""
    lines = [ln.strip() for ln in s.splitlines() if ln.strip()]
    return " ".join(lines).strip()


def parse_two_versions(text: str) -> dict:
    """
    반환: {"self": str|None, "counselor": str|None}
    Note/메타문이 뒤에 붙어도 counselor에 포함되지 않도록 컷.
    """
    if not isinstance(text, str):
        return {"self": None, "counselor": None}

    t = text.strip()

    # Note(또는 NOTE) 나오면 거기서부터 잘라내는 lookahead
    pat = r"SELF_VERSION\s*:\s*(.*?)\s*COUNSELOR_VERSION\s*:\s*(.*?)(?=\n\s*(?:Note|NOTE)\s*:|\Z)"
    m = re.search(pat, t, flags=re.IGNORECASE | re.DOTALL)
    if m:
        self_v = _squash(m.group(1))
        coun_v = _squash(m.group(2))
        return {"self": self_v or None, "counselor": coun_v or None}

    # 콜론 빠진 변형 fallback
    pat2 = r"SELF_VERSION\s*(.*?)\s*COUNSELOR_VERSION\s*(.*?)(?=\n\s*(?:Note|NOTE)\s*:|\Z)"
    m2 = re.search(pat2, t, flags=re.IGNORECASE | re.DOTALL)
    if m2:
        self_v = _squash(m2.group(1))
        coun_v = _squash(m2.group(2))
        return {"self": self_v or None, "counselor": coun_v or None}

    return {"self": None, "counselor": None}


def validate_and_fix(
    thought_text: str,
    cls: dict,
    draft_text: str,
    self_docs=None,
    counselor_docs=None,
    threshold: float = 0.7,
    top_k_labels: int = 3,
    max_regens: int = 2,
    regen_temperature: float = 0.4,
    max_tokens: int = 512,
) -> str:
    candidate = _cut_from_last_self_version(draft_text)

    for attempt in range(max_regens + 1):
        parsed = parse_two_versions(candidate)
        format_ok = bool(parsed["self"] and parsed["counselor"])

        if not format_ok:
            need_regen = True
            self_res = counselor_res = None
        else:
            self_res = classify_distortion(parsed["self"], threshold=threshold, top_k=top_k_labels)
            counselor_res = classify_distortion(parsed["counselor"], threshold=threshold, top_k=top_k_labels)
            need_regen = bool(self_res.get("is_distorted", True) or counselor_res.get("is_distorted", True))

        # 통과 시: 원문 candidate 말고 "재조립된 포맷"만 반환
        if not need_regen:
            return parse_two_versions(parsed["self"], parsed["counselor"])

        if attempt == max_regens:
            #  마지막에도 형식이 잡혀있다면, 최소한 포맷만 반환(노트 제거)
            if format_ok:
                return parse_two_versions(parsed["self"], parsed["counselor"])
            return candidate

        # docs 없으면 원문 기반으로 다시 retrieval
        if self_docs is None or counselor_docs is None:
            _self_docs, _counselor_docs = retrieve_two_styles(thought_text, cls, top_k_self=3, top_k_counselor=3)
        else:
            _self_docs, _counselor_docs = self_docs, counselor_docs

        #  4) 재생성 프롬프트(형식 강제 + 메타문 금지)
        extra = (
            "\n[REWRITE INSTRUCTIONS]\n"
            "The previous output is invalid or still distorted.\n"
            "Do NOT write any preface, explanation, or notes.\n"
            "Output ONLY the two sections in the exact format below.\n\n"
            "OUTPUT FORMAT (must be exact):\n"
            "SELF_VERSION:\n"
            "<one paragraph>\n\n"
            "COUNSELOR_VERSION:\n"
            "<one paragraph>\n"
        )

        messages = build_messages_two_versions(thought_text, cls, _self_docs, _counselor_docs)
        messages[-1]["content"] = messages[-1]["content"] + extra

        regenerated = generate_with_llama(
            messages,
            max_tokens=max_tokens,
            temperature=regen_temperature,
            top_p=0.9,
        )
        candidate = _cut_from_last_self_version(regenerated)

    return candidate


In [ ]:
def run_pipeline_two_versions(
    thought_text: str,
    top_k_self: int = 3,
    top_k_counselor: int = 3,
):
    """
    1) classify_distortion: LoRA+DANN + SMOTE+LoRA 분류기 사용
    2) retrieve_two_styles: RAG로 self / counselor 문장 검색
    3) build_messages_two_versions: Llama 프롬프트 만들기
    4) generate_with_llama: 실제 대안적 사고 두 버전 생성
    5) validate_and_fix: (지금은 그대로 통과)

    반환:
      {
        "classifier": cls,
        "self_docs": self_docs,
        "counselor_docs": counselor_docs,
        "draft": draft,
        "final": final,
      }
    """

    # 1) 자동적 사고 분류
    cls = classify_distortion(thought_text)

    # 2) RAG 검색
    self_docs, counselor_docs = retrieve_two_styles(
        thought_text,
        cls,
        top_k_self=top_k_self,
        top_k_counselor=top_k_counselor,
    )

    # 3) Llama 프롬프트 생성
    messages = build_messages_two_versions(
        thought_text,
        cls,
        self_docs,
        counselor_docs,
    )

    # 4) 생성
    draft = generate_with_llama(messages)

    # 5) 검증/수정: 자르지 말고 draft 그대로 넘기기 + docs도 넘기기
    final = validate_and_fix(
        thought_text, cls, draft,
        self_docs=self_docs, counselor_docs=counselor_docs
    )



    return {
        "classifier": cls,
        "self_docs": self_docs,
        "counselor_docs": counselor_docs,
        "draft": draft,
        "final": final,
    }


## 실행

In [ ]:
test_text = input()

result = run_pipeline_two_versions(test_text, top_k_self=3, top_k_counselor=3)

print("=== Result ===")
print("Text:", test_text)
print(f"{'Has Distortion' if result['classifier']['is_distorted'] else 'No Distortion'}")
print(f"p(distorted) = {result['classifier']['prob_distorted']:.3f}")

print("\nTop Distortion Labels:")
for name, score in result["classifier"]["top_labels"]:
    print(f" - {name}: {score:.3f}")

'''
print("\n=== RAG Retrieved Examples (SELF_VERSION candidates) ===")
for i, d in enumerate(result["self_docs"], 1):
    meta = d.get("meta", {})
    print(f"[SELF {i}] score={d.get('score', 0.0):.3f}, match_count={d.get('match_count', 0)}, type={d.get('reframe_type')}")
    print("  situation:", meta.get("situation"))
    print("  thought  :", meta.get("thought"))
    print("  reframe  :", d.get("support"))
    print()

print("=== RAG Retrieved Examples (COUNSELOR_VERSION candidates) ===")
for i, d in enumerate(result["counselor_docs"], 1):
    meta = d.get("meta", {})
    print(f"[COUNSELOR {i}] score={d.get('score', 0.0):.3f}, match_count={d.get('match_count', 0)}, type={d.get('reframe_type')}")
    print("  situation:", meta.get("situation"))
    print("  thought  :", meta.get("thought"))
    print("  reframe  :", d.get("support"))
    print()
'''

print("\n=== Alternative Thought by LLM ===")
print(result["final"])




I am anxious because it is almost my turn for the speech


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 259 prefix-match hit, remaining 841 prompt tokens to eval
llama_perf_context_print:        load time =    1531.78 ms
llama_perf_context_print: prompt eval time =     498.17 ms /   841 tokens (    0.59 ms per token,  1688.18 tokens per second)
llama_perf_context_print:        eval time =     796.87 ms /    56 runs   (   14.23 ms per token,    70.28 tokens per second)
llama_perf_context_print:       total time =    1367.14 ms /   897 tokens
llama_perf_context_print:    graphs reused =         53
Llama.generate: 1095 prefix-match hit, remaining 67 prompt tokens to eval
llama_perf_context_print:        load time =    1531.78 ms
llama_perf_context_print: prompt eval time =      52.73 ms /    67 tokens (    0.79 ms per token,  1270.60 tokens per second)
llama_perf_context_print:        eval time =    1687.27 ms /   117 runs   (   14.42 ms per token,    69.34 tokens per second)
llama_perf_context_print:       total time =    1894.00 ms /   184 tokens
llama_perf_context_print: 

=== Result ===
Text: I am anxious because it is almost my turn for the speech
No Distortion
p(distorted) = 0.666

Top Distortion Labels:

=== Alternative Thought by LLM ===
SELF_VERSION:
I am feeling some nervousness, but I have given many presentations before and I have done well. I can take a few deep breaths and focus on my message to calm my nerves.

COUNSELOR_VERSION:
You are feeling anxious about giving a speech, but it's okay to feel nervous. You've prepared well and can draw on your past experiences to help you feel more confident. Take a few deep breaths and focus on your message to calm your nerves.
